## load wrong data

In [15]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
wrong_file = '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.txt'

In [3]:
with open(wrong_file,'r') as f:
    wrong_list = f.readlines()
len(wrong_list)

In [5]:
uid = []
iid = []
his = [] 
label = []
his_label = []
his_title = []
title = []
for line in wrong_list:
    data = line.split('\sep')
    uid.append(int(data[0]))
    iid.append(int(data[1]))
    his.append(eval(data[2]))
    label.append(int(data[3]))
    his_label.append(eval(data[4]))
    his_title.append(data[5].replace('with feature <ItemID> ',''))
    title.append(data[6].replace('\n',''))

In [6]:
wrong_data = pd.DataFrame({"uid":uid,'iid':iid,'label':label, 'his':his,'his_label':his_label,'his_title':his_title,'title':title})

In [7]:
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[20334, 10160, 9536, 37380, 15208, 19818]","[1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [8]:
wrong_data['his_pad'] = wrong_data['his'].map(lambda x: [0]+x if len(x) < 10 else x)
wrong_data['his_label_pad'] = wrong_data['his_label'].map(lambda x: [0]+x if len(x) < 10 else x)

In [9]:
wrong_data = wrong_data[['uid','iid','label','his_pad','his_label_pad','his_title','title']]
wrong_data.columns = ['uid','iid','label','his','his_label','his_title','title']
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[0, 20334, 10160, 9536, 37380, 15208, 19818]","[0, 1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [10]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Get analysis

In [2]:
import os
import numpy as np
import pandas as pd

In [56]:
wrong_data = pd.read_csv('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.csv')

In [57]:
from tqdm import tqdm
import requests
from openai import OpenAI
url = 'https://api.siliconflow.cn/v1/'
api_key = 'sk-aqjpdmbxfmztkooxohnyxcbngysqhpgmjwxojihwwzuxojnj'

def get_response(client,message):
    for delay_secs in (2**x for x in range(0, 10)):
        try:
            response = client.chat.completions.create(
                model="Qwen/QwQ-32B",
                messages=message,
                stream=False, 
                max_tokens=1024
            )
            content = response.choices[0].message.content
            reasoning_content = response.choices[0].message.reasoning_content
            return content, reasoning_content
        except openai.OpenAIError as e:
            randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
            sleep_dur = delay_secs + randomness_collision_avoidance
            print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
            time.sleep(sleep_dur)
            continue

In [58]:
label2text = {0:'No',1:'Yes'}
prompt_template = '''A user has given high ratings to the following books: <HisItemList>. Using all available information, make a prediction about whether the user would enjoy the book titled <TargetItemTitle>?'''
hints = 'The correct response is <answer>. Reflect on multiple aspects based on historical information and reanalyze to determine if the target item will be liked by users. Explain the reason for the oversight based on the previous analysis.'

In [62]:
reasons = []
sbar = tqdm(total = len(wrong_data))
sbar.set_description('Generating Reasoning')
for idx in range(len(wrong_data)):
    row = wrong_data.iloc[idx]
    label = row['label']
    his_seq = row['his_title']
    target = row['title']
    message = prompt_template.replace('<HisItemList>',his_seq).replace('<TargetItemTitle>',target)
    his_message = [
        {"role": "system","content": "You are a helpful recommender answer with Yes or No."},
        {"role": "user","content": message}
    ]
    client = OpenAI(base_url=url,api_key=api_key)
    answer,reasoning_content = get_response(client,his_message)
    cnt = 1
    while label2text[label] not in answer:
        # his_message.pop(0)
        his_message.append({"role": "assistant","content": label2text[1-label]})
        his_message.append({"role": "user","content": hints.replace('<answer>',label2text[label])})
        answer,reasoning_content = get_response(client,his_message)
        cnt += 1
    reasons.append(f'{label2text[label]}. {reasoning_content}')
    sbar.set_postfix(cnt=cnt)
    if cnt > 5:
        print(f'cnt:{cnt}')
    sbar.update(1)
sbar.close()

Generating Reasoning:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Reasoning: 100%|██████████| 2/2 [01:14<00:00, 37.32s/it, cnt=2]


In [ ]:
wrong_data['reason'] = reasons

In [ ]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Merge csv

In [5]:
import os
import pandas as pd
import numpy as np
from glob import glob

In [2]:
csv_dir = '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection'

In [8]:
csv_files = glob(f'{csv_dir}/wrong1*.csv')

In [9]:
csv_files

['/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_15.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_6.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_7.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_14.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_17.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_3.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_4.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_8.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_10.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_2.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong1_batch_9.csv',
 '/home/yuqihang/projects/CoLLM/collm-datasets/boo

In [10]:

# csv_files = ['wrong_1.csv','wrong_2.csv']
save_file = 'wrong.csv'

In [11]:
dfs = []
for csv_file in csv_files:
    file_path = csv_file#os.path.join(csv_dir, csv_file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if not dfs:
            dfs.append(df)
        else:
            if list(df.columns) == list(dfs[0].columns):
                dfs.append(df)
            else:
                print(f"表头不一致: {csv_file}")
    else:
        print(f"文件不存在: {csv_file}")
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    print("没有有效的文件可以合并")

In [16]:
def convert_str_2_list(x):
    return np.array([int(item) for item in re.split(r'\s+', x[1:-1].replace(',',' ').strip(' '))])
def dftype(df):
    for idx in df.columns:
        if isinstance(df[idx][0],list) or isinstance(df[idx][0],np.ndarray):
            print(idx,type(df[idx][0]),type(df[idx][0][0]))
        else:
            print(idx,type(df[idx][0]))

In [17]:
df['his_label'] = df['his_label'].map(lambda x: convert_str_2_list(x))
df['his'] = df['his'].map(lambda x: convert_str_2_list(x))

In [19]:
dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


In [18]:
df.head()

,uid,iid,label,his,his_label,his_title,title,reason
0,6499,36528,1,"[44615, 42128, 42138, 44666, 31597, 45289, 345...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0]","""Absinthe"" (evaluation: Yes) with description:...","""The Scotch Queen (Volume 2)"" with description...","Yes. Okay, let's see. The user has rated sever..."
1,766,27516,1,"[27577, 24274, 27600, 28509, 32022, 27658, 153...","[1, 0, 0, 0, 1, 0, 1, 1, 1, 1]","""Can't Stop Loving You"" (evaluation: Yes) with...","""It's In His Arms (A Red River Valley Novel)"" ...","Yes. Okay, let's tackle this recommendation. T..."
2,11243,13856,0,"[0, 12398, 4899, 14876, 13883]","[0, 0, 1, 1, 1]","""Just a Kiss (A Summer Harbor Novel)"" (evaluat...","""An Uncommon Courtship (Hawthorne House)"" with...","No. Okay, let me try to figure this out. The u..."
3,9874,37781,1,"[13366, 21248, 14235, 23834, 12693, 20765, 386...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 0]","""Miracles We Have Seen: America's Leading Phys...","""Doughnut Economics: Seven Ways to Think Like ...","Yes. Okay, let me try to work through this ste..."
4,13010,26645,0,"[0, 17206, 46853, 25778, 27714, 17265]","[0, 1, 0, 1, 1, 1]","""Just for Now: Escape to New Zealand Book Thre...","""When Love Comes: A Diamond Creek, Alaska Nove...","No. Okay, let's tackle this again. The user ha..."


In [20]:
if len(save_file) == 0:
    save_ids = [csv_file.split('.')[0].split('_')[1:] for csv_file in csv_files]
    save_path = f'{csv_dir}/wrong_{"_".join(sorted(sum(save_ids,[])))}.csv'
else:
    save_path = f'{csv_dir}/{save_file}'
if os.path.exists(save_path):
    print(f'{save_path} already exists, skip')
else:
    df.to_csv(save_path,index=False)
save_path,len(df)

('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong.csv',
 850)

In [21]:
ori_df = pd.read_csv(f'{csv_dir}/reason.csv')

In [23]:
ori_df['his_label'] = ori_df['his_label'].map(lambda x: convert_str_2_list(x))
ori_df['his'] = ori_df['his'].map(lambda x: convert_str_2_list(x))

In [25]:
df = pd.concat([ori_df,df], ignore_index=True)

In [26]:
len(df),dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


(4500, None)

## Convert to training data

In [29]:
import re
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/reason.csv')

In [31]:
train = pd.read_pickle('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reason_ood2.pkl')

In [32]:
train.shape

(3650, 7)

In [33]:
len(df)

3650

In [5]:
cnt = 0
for idx in range(len(train)):
    if train['not_cold'].iloc[idx] == 1:
        cnt += 1
print(cnt)

650865


In [16]:
train.head(2)

,uid,iid,label,timestamp,his,flag,his_label,not_cold
0,1,4114,1,1.483920e+09,[0],-1.0,[0],1
1,1,34136,1,1.484698e+09,"[0, 4114]",-1.0,"[0, 1]",1


In [27]:
df.head(2)

,uid,iid,label,his,his_label,his_title,title,reason
0,4366,27528,0,"[6845, 18938, 42550, 15595, 19165, 25647, 3399...","[1, 1, 1, 1, 0, 1, 0, 1, 1, 0]","""Call to Honor: An Anthology (A Team Poseidon ...","""Tangled Up (Taking Chances)"" with description...","No. Okay, I need to reevaluate why the initial..."
1,358,34004,0,"[16302, 40934, 27313, 12594, 41187, 41331, 412...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","""The Sixth Window"" (evaluation: Yes) with desc...","""Silent Child"" with description: family, ameri...","No. Okay, let's tackle this recommendation pro..."


In [28]:
len(df)

4500

In [29]:
df['not_cold'] = pd.DataFrame(np.ones(df.shape[0]),index=df.index).astype("int")

In [30]:
wrong = df[['uid','iid','label','his','his_label','not_cold','reason']]

In [32]:
wrong.head(2)

,uid,iid,label,his,his_label,not_cold,reason
0,4366,27528,0,"[6845, 18938, 42550, 15595, 19165, 25647, 3399...","[1, 1, 1, 1, 0, 1, 0, 1, 1, 0]",1,"No. Okay, I need to reevaluate why the initial..."
1,358,34004,0,"[16302, 40934, 27313, 12594, 41187, 41331, 412...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]",1,"No. Okay, let's tackle this recommendation pro..."


In [33]:
wrong['his_label'] = wrong['his_label'].map(lambda x: convert_str_2_list(x))
wrong['his'] = wrong['his'].map(lambda x: convert_str_2_list(x))

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [34]:
dftype(wrong)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
not_cold <class 'numpy.int64'>
reason <class 'str'>


In [35]:
len(wrong)

4500

In [36]:
wrong.to_pickle('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reason_ood2.pkl')

In [ ]:
# ['uid','iid','his','label'] not_cold his_label